In [1]:
import pandas as pd
import numpy as np
import pickle
pd.set_option('display.max_columns', 100)

In [2]:
pwd

'/Users/jmirabito/Desktop/Learn.co/mod_3_bakeoff/Bakeoff2'

In [3]:
df = pd.read_csv('/Users/jmirabito/Desktop/Learn.co/nyc-mhtn-ds-080320-lectures/Mod_3/bake_off/holdout_data.csv')

In [4]:
df_columns = pd.read_csv('training_data.csv' , index_col=0)
df_columns.drop(['Y'], axis=1, inplace=True)

In [5]:
df_columns.reset_index(inplace=True)

In [6]:
print(len(df.columns))
print(len(df_columns.columns))

24
24


In [7]:
# Rename columns
df.columns=list(df_columns.iloc[18381])
# Remove row where ID = 'ID'
df=df[df.ID!="ID"]

df.reset_index(drop=True,inplace=True)
# Rename target variable as Y
df=df.rename(columns = {'default payment next month':'Y'})

df=df.astype(int)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


In [8]:
# Let's re-classify marriage status = 0 as other (3)
df.MARRIAGE = df.MARRIAGE.apply(lambda x: 3 if x==0 else x)

In [9]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,5501,180000,2,2,1,44,0,0,0,0,0,0,161186,167080,170788,174764,162667,166953,10000,8000,7000,6000,7000,10000
1,28857,130000,2,2,1,48,-2,-2,-2,-2,-2,-2,0,1240,1487,1279,749,440,1240,1487,1279,749,440,849
2,11272,60000,2,1,1,43,-1,3,2,0,0,-1,495,330,495,330,165,340,0,330,0,0,340,0
3,8206,240000,1,1,1,42,0,0,0,0,0,0,72339,91045,91027,51508,51127,0,20000,2213,1030,1023,6790,10893
4,6362,100000,2,2,1,28,2,0,0,0,0,2,73073,74739,70844,63924,57326,59654,3500,3003,1910,2400,3300,0


In [10]:
# Re-classifying all non-married persons as single
df.MARRIAGE = df.MARRIAGE.apply(lambda x: 2 if x==3 else x)

In [11]:
# Grouping 5,6, and 0 into unknown (5)
conditions = [
    df.EDUCATION==0,
    df.EDUCATION==6
]

choices = [
    5,
    5
]

df.EDUCATION = np.select(conditions, choices, default=df.EDUCATION)

In [12]:
# Binning Age
conditions = [
    (df.AGE>=20)&(df.AGE<30),
    (df.AGE>=30)&(df.AGE<40),
    (df.AGE>=40)&(df.AGE<50),
    (df.AGE>=50)&(df.AGE<60),
    (df.AGE>=60)&(df.AGE<70),
    (df.AGE>=70)&(df.AGE<80)
]

choices = [
    '20s',
    '30s',
    '40s',
    '50s',
    '60s',
    '70s'
]

df.AGE = np.select(conditions, choices)

In [13]:
# Gen feature for num months that went over credit limit
df['MON_OV_BAL']=0
conditions=[
    df.BILL_AMT1>=df.LIMIT_BAL,
    df.BILL_AMT2>=df.LIMIT_BAL,
    df.BILL_AMT3>=df.LIMIT_BAL,
    df.BILL_AMT4>=df.LIMIT_BAL,
    df.BILL_AMT5>=df.LIMIT_BAL,
    df.BILL_AMT6>=df.LIMIT_BAL,
]
choises = [
    df['MON_OV_BAL']+1,
    df['MON_OV_BAL']+1,
    df['MON_OV_BAL']+1,
    df['MON_OV_BAL']+1,
    df['MON_OV_BAL']+1,
    df['MON_OV_BAL']+1,
      
]
df['MON_OV_BAL']=np.select(conditions,choises)

In [14]:
# Column summing Pay columns
df['SUM_PAY_STAT']=df[['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']].sum(axis=1)
df['SUM_PAY_STAT']

0        0
1      -12
2        3
3        0
4        4
        ..
7496   -12
7497     0
7498    -9
7499     1
7500    -9
Name: SUM_PAY_STAT, Length: 7501, dtype: int64

In [15]:
# Feature for if bill amount increases
df['MON_BIL_INC']=0
conditions=[
    df.BILL_AMT2>=df.BILL_AMT1,
    df.BILL_AMT3>=df.BILL_AMT2,
    df.BILL_AMT4>=df.BILL_AMT3,
    df.BILL_AMT5>=df.BILL_AMT4,
    df.BILL_AMT6>=df.BILL_AMT5,
]
choises = [
    df['MON_BIL_INC']+1,
    df['MON_BIL_INC']+1,
    df['MON_BIL_INC']+1,
    df['MON_BIL_INC']+1,
    df['MON_BIL_INC']+1,
    
      
]
df['MON_BIL_INC']=np.select(conditions,choises,default=df['MON_BIL_INC']+0)

In [16]:
total=[]
for i in range(len(df)):
    total.append(list(df[['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']].loc[i].mode()))
total=[x[0]for x in total]
df['PAY_STAT_MODE']=total

In [17]:
df['BILL_AV']=df[['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']].mean(axis=1)

In [18]:
df['PAY_AV']=df[['PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']].mean(axis=1)

In [19]:
# Percentage of one's credit limit that is used
df['AVG_UTILIZATION'] = df.BILL_AV/df.LIMIT_BAL

In [20]:
# Average percentage of credit card bill paid
df['AVG_PERCENT_PAID'] = df.PAY_AV/df.BILL_AV

In [21]:
df.AVG_PERCENT_PAID.fillna(value=0, inplace=True)

In [22]:
df.AVG_PERCENT_PAID.replace(np.inf, value=0, inplace=True)

In [23]:
#Creating dummy variables for age group
df=pd.get_dummies(df,columns=['AGE'],drop_first=True)

In [24]:
#Creating a column that calculates the usage of the card.
df['PERCENT_USAGE']=(df[['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']].sum(axis=1)*600)/(df['LIMIT_BAL']*6)

In [25]:
df['BEEN_LATE']=0
conditions=[
(df.PAY_0>=1)|(df.PAY_2>=1)|(df.PAY_3>=1)|(df.PAY_4>=1)|(df.PAY_5>=1)|(df.PAY_6>=1),

]
choises = [
    1,
]
df['BEEN_LATE']=np.select(conditions,choises,default=0)   
      

In [26]:
df['TOTAL_USAGE']=(df[['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']].sum(axis=1))

In [27]:
df['TOTAL_PAY']=(df[['PAY_AMT1','PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']].sum(axis=1))

In [28]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,MON_OV_BAL,SUM_PAY_STAT,MON_BIL_INC,PAY_STAT_MODE,BILL_AV,PAY_AV,AVG_UTILIZATION,AVG_PERCENT_PAID,AGE_30s,AGE_40s,AGE_50s,AGE_60s,AGE_70s,PERCENT_USAGE,BEEN_LATE,TOTAL_USAGE,TOTAL_PAY
0,5501,180000,2,2,1,0,0,0,0,0,0,161186,167080,170788,174764,162667,166953,10000,8000,7000,6000,7000,10000,0,0,1,0,167239.666667,8000.000000,0.929109,0.047836,0,1,0,0,0,557.465556,0,1003438,48000
1,28857,130000,2,2,1,-2,-2,-2,-2,-2,-2,0,1240,1487,1279,749,440,1240,1487,1279,749,440,849,0,-12,1,-2,865.833333,1007.333333,0.006660,1.163426,0,1,0,0,0,3.996154,0,5195,6044
2,11272,60000,2,1,1,-1,3,2,0,0,-1,495,330,495,330,165,340,0,330,0,0,340,0,0,3,1,-1,359.166667,111.666667,0.005986,0.310905,0,1,0,0,0,3.591667,1,2155,670
3,8206,240000,1,1,1,0,0,0,0,0,0,72339,91045,91027,51508,51127,0,20000,2213,1030,1023,6790,10893,0,0,1,0,59507.666667,6991.500000,0.247949,0.117489,0,1,0,0,0,148.769167,0,357046,41949
4,6362,100000,2,2,1,2,0,0,0,0,2,73073,74739,70844,63924,57326,59654,3500,3003,1910,2400,3300,0,0,4,1,0,66593.333333,2352.166667,0.665933,0.035321,0,0,0,0,0,399.560000,1,399560,14113


## Predicting Default

In [29]:
ls

2020_09_22_classification_bakeoff_modeling.ipynb
bakeoff2.csv
bakeoff2_predictions_JMIR.ipynb
bakeoff_instructions.md
dt_poly_GS.pickle
final_dt_model.pickle
holdout_data.csv
holdout_predictions_JMIR.csv
ignacio-classification-bakeoff-template.ipynb
jamil-classification-bakeoff-template2.ipynb
metrics_df.pickle
model_perf_df.pickle
results_df.pickle
training_data.csv
winning_model.pickle


In [30]:
model = pd.read_pickle(r'winning_model.pickle')

In [32]:
model_predictions = (model.predict_proba(df)[:,1] >= 0.663).astype(bool)

In [33]:
pd.DataFrame(model_predictions).to_csv("holdout_predictions_JMIR.csv")